# Experiment with Stemmers

Some stemmers tend to be better for certain domains. Do not forget to experiment with different stemmers.

Here is a comparison between *Porter* and *Snowball* stemmers that come with NLTK.

In [17]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

# Let's look at the languages supported by Snowball
print(" ".join(SnowballStemmer.languages))

stemmer1 = PorterStemmer()
stemmer2 = SnowballStemmer("english")

tokens = ['caresses', 'flies', 'dies', 'mules', 'denied', 
           'died', 'agreed', 'owned', 'humbled', 'sized']

print(' '.join([stemmer1.stem(t) for t in tokens]))
print(' '.join([stemmer2.stem(t) for t in tokens]))

danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish
caress fli die mule deni die agre own humbl size
caress fli die mule deni die agre own humbl size


Almost identical results! Now let's try a difficult case - *generously* and a few of its misspelings.

In [40]:
tokens = ['generously', 'generous', 'generousness', 'overgenerous',
          'generosly', 'generusly', 'genorously', 'genera', 'generally']
print(' '.join([stemmer1.stem(t) for t in tokens]))
print(' '.join([stemmer2.stem(t) for t in tokens]))

gener gener gener overgener generosli generusli genor genera gener
generous generous generous overgener generosli generusli genor genera general


As can be seen, while the performance of both the stemmers are the same for the misspelt words. The *Snowball* stemmer performs better on the original word and its variations.

# More features with Topic Modeling

    
## What will we be doing?

* We'll walk through a basic application of Topic Modeling with LDA
* We'll also cover the basic NLP operations necessary for the application
    

## Let's create some data

In [3]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

## Let's fast-forward through pre-processing

* After the processing, we'll have *texts* - a tokenized, stopped and stemmed list of words from a single document
* Let’s fast forward and loop through all our documents and appended each one to *texts*
* So now *texts* is a list of lists, one list for each of our original documents

In [4]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from pprint import pprint

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

print("\n##### texts")
print(texts)

print("\n##### The lines in texts")
for line in texts:
    print(line)

Using TensorFlow backend.



##### texts
[['brocolli', 'good', 'eat', 'brother', u'like', 'eat', 'good', 'brocolli', 'mother'], ['mother', u'spend', 'lot', 'time', u'drive', 'brother', 'around', u'basebal', u'practic'], ['health', u'expert', 'suggest', u'drive', 'may', u'caus', u'increas', 'tension', 'blood', u'pressur'], ['often', 'feel', u'pressur', 'perform', 'well', 'school', 'mother', 'never', u'seem', 'drive', 'brother', 'better'], ['health', u'profession', 'say', 'brocolli', 'good', 'health']]

##### The lines in texts
['brocolli', 'good', 'eat', 'brother', u'like', 'eat', 'good', 'brocolli', 'mother']
['mother', u'spend', 'lot', 'time', u'drive', 'brother', 'around', u'basebal', u'practic']
['health', u'expert', 'suggest', u'drive', 'may', u'caus', u'increas', 'tension', 'blood', u'pressur']
['often', 'feel', u'pressur', 'perform', 'well', 'school', 'mother', 'never', u'seem', 'drive', 'brother', 'better']
['health', u'profession', 'say', 'brocolli', 'good', 'health']


## What's next?

* To generate an LDA model, we need to understand how frequently each term occurs within each document
* To do that, we need to construct a document-term matrix with a package called *gensim*

# Topic Modeling with gensim

## Getting started with gensim?




In [5]:
from gensim import corpora, models

dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(32 unique tokens: [u'often', u'feel', u'profession', u'drive', u'say']...)


* The Dictionary() function traverses texts, assigning a unique integer id to each unique token while also collecting word counts and relevant statistics
* To see each token’s unique integer id, try -

In [6]:
print(dictionary.token2id)

{u'often': 23, u'feel': 24, u'profession': 31, u'drive': 8, u'say': 30, u'pressur': 18, u'basebal': 7, u'seem': 29, u'expert': 14, u'perform': 28, u'suggest': 16, u'better': 27, u'health': 19, u'lot': 9, u'tension': 13, u'good': 1, u'around': 6, u'may': 15, u'mother': 4, u'school': 22, u'blood': 20, u'never': 25, u'increas': 21, u'eat': 5, u'practic': 12, u'brocolli': 0, u'like': 2, u'well': 26, u'brother': 3, u'caus': 17, u'time': 10, u'spend': 11}


Next, our dictionary must be converted into a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) -

In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2)], [(3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(3, 1), (4, 1), (8, 1), (18, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)], [(0, 1), (1, 1), (19, 2), (30, 1), (31, 1)]]


In [8]:
print(corpus[0])

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2)]


In [9]:
print(corpus[1])

[(3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]


In [10]:
for line in corpus:
    print(line)

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2)]
[(3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
[(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)]
[(3, 1), (4, 1), (8, 1), (18, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]
[(0, 1), (1, 1), (19, 2), (30, 1), (31, 1)]


* The doc2bow() function converts dictionary into a bag-of-words
* The result, *corpus*, is a list of vectors equal to the number of documents
* In each document vector is a series of tuples
* The tuples are (term ID, term frequency) pairs
* This includes terms that actually occur - terms that do not occur in a document will not appear in that document’s vector

## A Quick Quiz

Looking at the data above, please answer the following:
* How many times does *basebal* occur in *doc_a*?
* How many times does *basebal* occur in *doc_b*?
* How many times does *health* occur in *doc_e*?
* Give an example of a word that occurs in *doc_a* but doesn't occur in *doc_b*.
* How many times does *brother* occur in all the documents?

## Creating the LDA Model

*corpus* is a (sparse) document-term matrix and now we’re ready to generate an LDA model

In [11]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

## Parameters to the LDA model

https://radimrehurek.com/gensim/models/ldamodel.html
* num_topics
    - required
    - An LDA model requires the user to determine how many topics should be generated
    - Our document set is small, so we’re only asking for three topics
* id2word
    - required
    - The LdaModel class requires our previous dictionary to map ids to strings
* passes
    - optional
    - The number of laps the model will take through corpus
    - The greater the number of passes, the more accurate the model will be
    - A lot of passes can be slow on a very large corpus.

In [12]:
print(ldamodel)

LdaModel(num_terms=32, num_topics=3, decay=0.5, chunksize=2000)


In [13]:
print(ldamodel.print_topics())

[(0, u'0.082*"brocolli" + 0.082*"good" + 0.081*"brother" + 0.081*"mother" + 0.081*"eat" + 0.046*"lot" + 0.046*"time" + 0.046*"basebal" + 0.046*"practic" + 0.046*"around"'), (1, u'0.125*"health" + 0.050*"increas" + 0.050*"may" + 0.050*"blood" + 0.050*"suggest" + 0.050*"caus" + 0.050*"tension" + 0.050*"expert" + 0.050*"say" + 0.050*"profession"'), (2, u'0.059*"drive" + 0.059*"pressur" + 0.059*"perform" + 0.059*"school" + 0.059*"often" + 0.059*"never" + 0.059*"well" + 0.059*"feel" + 0.059*"better" + 0.059*"seem"')]


In [14]:
for topic in ldamodel.print_topics(num_topics=2):
    print(topic)

(0, u'0.082*"brocolli" + 0.082*"good" + 0.081*"brother" + 0.081*"mother" + 0.081*"eat" + 0.046*"lot" + 0.046*"time" + 0.046*"basebal" + 0.046*"practic" + 0.046*"around"')
(2, u'0.059*"drive" + 0.059*"pressur" + 0.059*"perform" + 0.059*"school" + 0.059*"often" + 0.059*"never" + 0.059*"well" + 0.059*"feel" + 0.059*"better" + 0.059*"seem"')


In [15]:
for topic in ldamodel.print_topics(num_topics=3, num_words=3):
    print(topic)

(0, u'0.082*"brocolli" + 0.082*"good" + 0.081*"brother"')
(1, u'0.125*"health" + 0.050*"increas" + 0.050*"may"')
(2, u'0.059*"drive" + 0.059*"pressur" + 0.059*"perform"')


* Within each topic are the three most probable words to appear in that topic

## Topics in detail
Let's now look at a topic in detail. Let us see how distinct the topics are, and if they seem to capture any context.

In [16]:
print(ldamodel.print_topic(topicno=0))

0.082*"brocolli" + 0.082*"good" + 0.081*"brother" + 0.081*"mother" + 0.081*"eat" + 0.046*"lot" + 0.046*"time" + 0.046*"basebal" + 0.046*"practic" + 0.046*"around"


In [17]:
print(ldamodel.print_topic(topicno=1))

0.125*"health" + 0.050*"increas" + 0.050*"may" + 0.050*"blood" + 0.050*"suggest" + 0.050*"caus" + 0.050*"tension" + 0.050*"expert" + 0.050*"say" + 0.050*"profession"


In [18]:
print(ldamodel.print_topic(topicno=2))

0.059*"drive" + 0.059*"pressur" + 0.059*"perform" + 0.059*"school" + 0.059*"often" + 0.059*"never" + 0.059*"well" + 0.059*"feel" + 0.059*"better" + 0.059*"seem"


## Do the topics make sense?

In [19]:
for topic in ldamodel.print_topics(num_topics=3, num_words=3):
    print(topic)

(0, u'0.082*"brocolli" + 0.082*"good" + 0.081*"brother"')
(1, u'0.125*"health" + 0.050*"increas" + 0.050*"may"')
(2, u'0.059*"drive" + 0.059*"pressur" + 0.059*"perform"')


* Even though our document set is small the model is reasonable
* Third Topic - health, brocolli and good make sense together
* Second Topic includes mother and brother, which is reasonable
* First Topic - confusing!
    - If we revisit the original documents, we see that drive has multiple meanings
        * driving a car
        * driving oneself to improve
    - This is something to note
* Two topics seems like a better fit for our documents!

## Refining the model

Two topics seems like a better fit for our documents!

In [20]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

for topic in ldamodel.print_topics(num_topics=2, num_words=4):
    print(topic)

(0, u'0.109*"health" + 0.047*"pressur" + 0.047*"drive" + 0.047*"may"')
(1, u'0.076*"mother" + 0.076*"brother" + 0.055*"brocolli" + 0.055*"good"')


Let's try it with more passes:

In [21]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=200)

for topic in ldamodel.print_topics(num_topics=2, num_words=4):
    print(topic)

(0, u'0.076*"brother" + 0.076*"mother" + 0.055*"brocolli" + 0.055*"good"')
(1, u'0.109*"health" + 0.047*"pressur" + 0.047*"drive" + 0.047*"expert"')


# The Final Topic Modeling Script

In [24]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

# print the topics
for topic in ldamodel.print_topics(num_topics=2, num_words=4):
    print(topic)

(0, u'0.054*"good" + 0.054*"brocolli" + 0.053*"drive" + 0.053*"brother"')
(1, u'0.082*"health" + 0.048*"mother" + 0.048*"brother" + 0.048*"drive"')


## Predicting Topic for new documents

In [35]:
doc_f = "Are Health professionals justified in saying that brocolli is good for your health?" 

doc_set = [doc_f]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

infer = ldamodel[corpus[0]]

# https://radimrehurek.com/gensim/wiki.html

In [36]:
infer

[(0, 0.92344832516970621), (1, 0.076551674830293848)]